In [19]:
!nvidia-smi

Tue Jun  7 16:52:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   45C    P0    26W /  70W |   8832MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

In [3]:
path = ''

In [4]:
train_png = sorted(glob('/home/lab17/jupyter_home/Data/bdata_snacks/*.jpg'))

# files = os.listdir('/home/lab17/jupyter_home/Data/bdata_snacks')
# files

In [5]:
len(train_png), len(set(train_png))

(311, 311)

In [6]:
files = os.listdir('/home/lab17/jupyter_home/Data/bdata_snacks')
train_labels = sorted(files)
train_labels

['(G) 노브랜드쿠키앤크림샌드720g.jpg',
 '(G)[피코크] 초코칩쿠키 브라우니 150g.jpg',
 '(G)프링글스 사워크림앤어니언 134g.jpg',
 '(G)프링글스 오리지널 134g.jpg',
 '[ACT2]팝콘 234g(3입).jpg',
 '[농심] 감자깡 250g.jpg',
 '[농심] 감자깡 75g.jpg',
 '[농심] 고구마깡 83g.jpg',
 '[농심] 꿀꽈배기 90g.jpg',
 '[농심] 닭다리 후라이드치킨맛 66g.jpg',
 '[농심] 매운새우깡 90g.jpg',
 '[농심] 매운새우깡400G.jpg',
 '[농심] 새우깡 400g.jpg',
 '[농심] 새우깡90g.jpg',
 '[농심] 수미칩 오리지널 85g.jpg',
 '[농심] 쌀새우깡 80g.jpg',
 '[농심] 알새우칩 68g.jpg',
 '[농심] 인디안밥 83g.jpg',
 '[농심] 자갈치90g.jpg',
 '[농심] 칩포테토 오리지날 125g.jpg',
 '[농심] 포스틱 84g.jpg',
 '[롯데] 꼬깔콘 고소한맛 67g.jpg',
 '[롯데] 꼬깔콘 군옥수수맛 67g.jpg',
 '[롯데] 쌀로별 오리지널 156g.jpg',
 '[롯데]도리토스 갈비천왕치킨맛172G.jpg',
 '[리코스] 라운드 나쵸칩 454g.jpg',
 '[보겔] 팝콘용 옥수수 700g.jpg',
 '[삼양] 사또밥 85g.jpg',
 '[삼양] 짱구 272g.jpg',
 '[아미고] 나쵸칩 454g.jpg',
 '[오리온]  고래밥 볶음 양념 160G.jpg',
 '[오리온]  눈을 감자 113g.jpg',
 '[오리온]  오감자 115g.jpg',
 '[오리온] 도도한나쵸 155g.jpg',
 '[오리온] 스윙칩 볶음고추장 124g.jpg',
 '[오리온] 오징어 땅콩(오리지널) 202g.jpg',
 '[오리온] 왕고래밥 56g.jpg',
 '[오리온] 포카칩 어니언맛 66g.jpg',
 '[오리온] 포카칩 오리지널 66g.jpg',
 '[크라운] 꽃게랑 143g.jpg',

In [7]:
train_dict = {i:l[:-4] for i, l in zip(range(311), train_labels)}
train_dict

{0: '(G) 노브랜드쿠키앤크림샌드720g',
 1: '(G)[피코크] 초코칩쿠키 브라우니 150g',
 2: '(G)프링글스 사워크림앤어니언 134g',
 3: '(G)프링글스 오리지널 134g',
 4: '[ACT2]팝콘 234g(3입)',
 5: '[농심] 감자깡 250g',
 6: '[농심] 감자깡 75g',
 7: '[농심] 고구마깡 83g',
 8: '[농심] 꿀꽈배기 90g',
 9: '[농심] 닭다리 후라이드치킨맛 66g',
 10: '[농심] 매운새우깡 90g',
 11: '[농심] 매운새우깡400G',
 12: '[농심] 새우깡 400g',
 13: '[농심] 새우깡90g',
 14: '[농심] 수미칩 오리지널 85g',
 15: '[농심] 쌀새우깡 80g',
 16: '[농심] 알새우칩 68g',
 17: '[농심] 인디안밥 83g',
 18: '[농심] 자갈치90g',
 19: '[농심] 칩포테토 오리지날 125g',
 20: '[농심] 포스틱 84g',
 21: '[롯데] 꼬깔콘 고소한맛 67g',
 22: '[롯데] 꼬깔콘 군옥수수맛 67g',
 23: '[롯데] 쌀로별 오리지널 156g',
 24: '[롯데]도리토스 갈비천왕치킨맛172G',
 25: '[리코스] 라운드 나쵸칩 454g',
 26: '[보겔] 팝콘용 옥수수 700g',
 27: '[삼양] 사또밥 85g',
 28: '[삼양] 짱구 272g',
 29: '[아미고] 나쵸칩 454g',
 30: '[오리온]  고래밥 볶음 양념 160G',
 31: '[오리온]  눈을 감자 113g',
 32: '[오리온]  오감자 115g',
 33: '[오리온] 도도한나쵸 155g',
 34: '[오리온] 스윙칩 볶음고추장 124g',
 35: '[오리온] 오징어 땅콩(오리지널) 202g',
 36: '[오리온] 왕고래밥 56g',
 37: '[오리온] 포카칩 어니언맛 66g',
 38: '[오리온] 포카칩 오리지널 66g',
 39: '[크라운] 꽃게랑 143g',
 40: '[크라

In [8]:
np.array(train_dict.keys())

array(dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218

In [9]:
np.array(list(train_dict.keys()))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [10]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [11]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
# test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|█████████████████████████████████████████| 311/311 [00:12<00:00, 24.38it/s]


In [32]:
np.save('/home/lab17/jupyter_home/Data/bdata_snacks_384.npy', np.array(train_imgs))

In [12]:
train_imgs = np.load('/home/lab17/jupyter_home/Data/bdata_snacks_384.npy')

In [13]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

# train 평균 0.4330380901867049 0.4034575319032911 0.39415050509784405
# train 표준편차 0.1815717110252788 0.17403455556798705 0.16323395055036488

train 평균 0.7097844470485838 0.633531969760679 0.5547763538712672
train 표준편차 0.2033680463341466 0.21580010480417353 0.24844798400637713


In [14]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
            # print(idx, '7')
            train_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(mean = [0.709784, 0.633531, 0.554776],
                                        std = [0.203368, 0.215800, 0.248448]),
                    transforms.RandomAffine((-45, 45)),

                    # transforms.RandomVerticalFlip(p=0.5),   # - 이미지를 랜덤으로 수직으로 뒤집는다. p =0이면 뒤집지 않는다.
                    # transforms.RandomHorizontalFlip(p=0.5), # - 이미지를 랜덤으로 수평으로 뒤집는다.
                    transforms.RandomRotation((0,80))       #  이미지를 랜덤으로 degrees 각도로 회전한다.

                ])
            img = train_transform(img)
#         if self.mode == 'test':
#           test_transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
#                                      std = [0.195055, 0.190053, 0.185323]),
              
#             ])
#           img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=311, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=311, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [18]:
def score_function(real, pred):
#     score = f1_score(real, pred, average="macro")
    score = accuracy_score(real, pred)
    return score

In [16]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [21]:
import gc

# cv = StratifiedKFold(n_splits = 5, random_state = 2022,shuffle=True)
batch_size = 32
epochs = 70
pred_ensemble = []

# for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):

# # start
# t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
# t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(list(train_dict.keys())), mode='train')
# print(len(np.array(train_imgs)))
# print(np.array(train_dict.keys()))
# print(len(np.array(train_dict.keys())))

train_loader = DataLoader(train_dataset, shuffle=False, batch_size=batch_size)

# # Val
# val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), np.array([]), mode='test')
# val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)


gc.collect()
torch.cuda.empty_cache()
best=0

model = Network().to(device)

#   optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
optimizer = torch.optim.Adadelta(model.parameters())
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()  

best_f1 = 0
early_stopping = 0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
#     with torch.no_grad():
#         val_loss = 0 
#         val_pred = []
#         val_y = []
#         for batch in (val_loader):
#             x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
#             y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
#             with torch.cuda.amp.autocast():
#                 pred_val = model(x_val)
#             loss_val = criterion(pred_val, y_val)

#             val_loss += loss_val.item()/len(val_loader)
#             val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
#             val_y += y_val.detach().cpu().numpy().tolist()
#         val_f1 = score_function(val_y, val_pred)
#         if val_f1 > best_f1:
#             best_epoch = epoch
#             best_loss = val_loss
#             best_f1 = val_f1
#             early_stopping = 0

#             torch.save({'epoch':epoch,
#                         'state_dict':state_dict,
#                         'optimizer': optimizer.state_dict(),
#                         'scaler': scaler.state_dict(),
#                  }, path +'0513_best_model_{}.pth'.format(idx))
#             print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
#         else:
#             early_stopping += 1

#               # Early Stopping
#         if early_stopping == 20:
#             TIME = time.time() - start
#             print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
#             print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
#             print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
#             break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
#     print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

epoch : 1/70    time : 11s/741s
TRAIN    loss : 5.76680    f1 : 0.00000
epoch : 2/70    time : 11s/737s
TRAIN    loss : 5.63477    f1 : 0.03215
epoch : 3/70    time : 11s/731s
TRAIN    loss : 5.47266    f1 : 0.09968
epoch : 4/70    time : 11s/721s
TRAIN    loss : 5.23555    f1 : 0.21865
epoch : 5/70    time : 11s/711s
TRAIN    loss : 4.85156    f1 : 0.34084
epoch : 6/70    time : 11s/701s
TRAIN    loss : 4.13379    f1 : 0.26045
epoch : 7/70    time : 11s/693s
TRAIN    loss : 2.75869    f1 : 0.51447
epoch : 8/70    time : 11s/682s
TRAIN    loss : 1.60049    f1 : 0.66881
epoch : 9/70    time : 11s/673s
TRAIN    loss : 0.58726    f1 : 0.88424
epoch : 10/70    time : 11s/660s
TRAIN    loss : 0.17039    f1 : 0.97428
epoch : 11/70    time : 11s/650s
TRAIN    loss : 0.07064    f1 : 0.98714
epoch : 12/70    time : 11s/639s
TRAIN    loss : 0.04967    f1 : 0.98714
epoch : 13/70    time : 11s/625s
TRAIN    loss : 0.07174    f1 : 0.99035
epoch : 14/70    time : 11s/616s
TRAIN    loss : 0.04223    